In [1]:
import spacy
nlp = spacy.load('en_core_web_lg')

import numpy as np
import pandas as pd

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import OneClassSVM
svm = LinearSVC()

In [3]:
import re
dpf = pd.read_csv('Del-Pass.csv')

keys_list = dpf['kpattern'].tolist()
regex = re.compile(r'(' + '|'.join(keys_list) + r')') 

cus_list = ['remove','removed','cancelled', 'cancel' ,'deleted']
regey = re.compile(r'(' + '|'.join(cus_list) + r')') 

import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
additional =["thank you","please","hello","hi","advise","hin't","st","nd","rd","th","thank"]
stopwords = stopwords + additional
remov=[]
for ele in stopwords:
    matches = re.findall("n't",ele)
    matches2 = re.findall("'nt",ele)
    if len(matches)>0 or len(matches2)>0:
        remov.append(ele)
stopwords = [word for word in stopwords if word not in remov]

In [4]:
#segment_featurizer = SegmentFeaturizer()  # more on this below
from sklearn.base import BaseEstimator, TransformerMixin
class CustomLinguisticFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass   
  #  def fit(self, X, y=None):
    def fit(self, X, y):
        return self    
    def transform(self, X):       
        ref_corpus=[]
        for text in X:           
            trans = regex.sub(lambda m: m.group().replace(m.group(),"delpass "),text)
            trans_ref = regey.sub(lambda n: n.group().replace(n.group(),"delete "),trans)
             
            ref_corpus.append(trans_ref)   
        return ref_corpus


In [5]:
class Lemm(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass   
  #  def fit(self, X, y=None):
    def fit(self, X, y):
        return self    
    def transform(self, X):       
        lemmatized_list=[]
        for snt in X:
        #    print(snt)
            tokens = nlp(snt)
        #    for token in tokenized:
            filtered_sentence = [w.lemma_ for w in tokens if not w.text in stopwords]
        #                 lemm = token.lemma_ for token.text in token
        #             lemmatized_list.append(lemm)
            stri = ' '.join(filtered_sentence)
            lemmatized_list.append(stri)
        return lemmatized_list

In [6]:

class CusttransformFeat(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass   
  #  def fit(self, X, y=None):
    def fit(self, X, y):
        return self    
    def transform(self, X):       
        X = np.array(X)
        #vectorizer = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("train 1 class tfidf vocab.pkl", "rb")))
        vectorizer = TfidfVectorizer(vocabulary=pickle.load(open("BigVocabTfidf 1 class.pkl", "rb")))
        vec_train = vectorizer.fit_transform(X).toarray()
        #pickle.dump(vectorizer.vocabulary_,open("BigVocabTfidf 1 class.pkl","wb"))
        #print(vec_train)
        return vec_train
        

In [11]:
class SpacyVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 300
        pass
    def fit(self, X, y):
        return self

    def transform(self, X):
        # Doc.vector defaults to an average of the token vectors.
        # https://spacy.io/api/doc#vector
        lemmatized_list=[]
        for snt in X:
        #    print(snt)
            tokens = nlp(snt)
        #    for token in tokenized:
            filtered_sentence = [w.text for w in tokens if not w.text in stopwords]
        #                 lemm = token.lemma_ for token.text in token
        #             lemmatized_list.append(lemm)
            stri = ' '.join(filtered_sentence)
            lemmatized_list.append(stri)
        return [self.nlp(text).vector for text in lemmatized_list]
       

In [15]:
import pickle
loaded_model = pickle.load(open('OneClassTFIDF_V_0.2.pkl','rb'))
from joblib import load
loaded_model2 = load(filename = 'P&P Classifier V_0.1.joblib')

In [76]:
test_df = pd.read_csv('Test_Input.csv')
test_df["desc"] = test_df["short_des"] + '. ' + test_df["desc"]
txt = test_df['desc'][0]
inc_n = pd.Series([test_df['inc_num'][0]])

In [80]:
#
#txt = "can you remove staff account status and entitlements on this account E7476870 as keith no longer works for oxendales eire"
desc = re.sub('\w\d{7}', ' ', txt)
desc = re.sub('[^a-zA-Z]', ' ', desc)
desc = desc.lower()
desc = desc.split()
desc = ' '.join(desc)
lst1 = [desc]
y_pr = loaded_model.predict(lst1)
#crpt = classification_report(y, y_pred)
c_type = []
if y_pr == 1:
    print("Belongs to a set of classified problems")
    c_pr = loaded_model2.predict(lst1)
    if c_pr[0] == 1:
        c_type.append('P')
        print("Class Type: ", c_type[0])
        
    else:
        c_type.append('R')
        print("Class Type: ", c_type[0])
        
else:
    print("Doesn't belongs to a set of classified problems")
    c_type.append('None')

Belongs to a set of classified problems
Class Type:  R


In [83]:
inc_data = re.findall(r'\w\d{7}',txt)
inc_data = [', '.join(inc_data)]
tdf_resp = pd.DataFrame({"Inc_Num": inc_n,"Type": c_type, "Data": inc_data })

In [84]:
tdf_resp.to_csv("Test_Input_Resp.csv", index=False)
tdf_resp

,Inc_Num,Type,Data
0,INC0808628,R,E7476870
